# Hybrid Forecasting: Prophet + LSTM
This notebook builds a hybrid time series forecasting model that combines:

- **Prophet** for capturing trend and seasonality
- **LSTM** to model the residual errors Prophet cannot explain

It evaluates performance over multiple forecast horizons using:
- MAE, RMSE, R², MAPE, Accuracy (%)

In [ ]:
!pip install prophet openpyxl tensorflow

!pip install prophet openpyxl

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import os
import joblib
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping

SAVE_DIR = "hybrid_prophet_lstm_output"
os.makedirs(SAVE_DIR, exist_ok=True)


In [ ]:
df = pd.read_excel("usask.sec.min_short_v2.xlsx")
df.columns = ['minute', 'requests']
df['minute'] = pd.to_datetime(df['minute'], unit='m', origin='unix')
df.rename(columns={'minute': 'ds', 'requests': 'y'}, inplace=True)
df = df.sort_values('ds').reset_index(drop=True)


In [ ]:
# --- Dependency Setup and Variable Initialization ---
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

horizons = [1, 5, 10, 15]  # Define forecast lengths
best_loss = float('inf')
train_loss_per_unit = []


In [ ]:
# Fit full Prophet model
model = Prophet(
    growth='linear',
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=True,
    changepoint_prior_scale=0.05,
    seasonality_prior_scale=10.0,
    holidays_prior_scale=10.0,
    n_changepoints=25
)
model.add_seasonality(name='hourly', period=1, fourier_order=3)
model.fit(df)

# Forecast same length
future = model.make_future_dataframe(periods=0, freq='min')
forecast = model.predict(future)

# Calculate residuals
df['yhat'] = forecast['yhat']
df['residual'] = df['y'] - df['yhat']


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Use residuals for LSTM training
window_size = 30  # 30-minute window
residual_series = df['residual'].values.reshape(-1, 1)

scaler = MinMaxScaler()
res_scaled = scaler.fit_transform(residual_series)

X = []
y = []
for i in range(window_size, len(res_scaled) - 15):  # leave room for horizon
    X.append(res_scaled[i - window_size:i])
    y.append(res_scaled[i:i + 15].flatten())

X, y = np.array(X), np.array(y)


In [ ]:
# Track accuracy with loss during training
train_loss_per_unit = []

for units in [32, 64]:
    model_lstm = Sequential()
    model_lstm.add(LSTM(units, activation='tanh', input_shape=(X.shape[1], X.shape[2])))
    model_lstm.add(Dense(max(horizons)))
    model_lstm.compile(optimizer='adam', loss='mse')
    es = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

    history = model_lstm.fit(X, y_lstm, epochs=30, batch_size=32, verbose=1, callbacks=[es])

    # Store training loss per epoch
    train_loss_per_unit.append((units, history.history['loss']))

    if history.history['loss'][-1] < best_loss:
        best_loss = history.history['loss'][-1]
        best_model = model_lstm

        # Save the best model to disk
        best_model.save(f"{SAVE_DIR}/best_lstm_model_units_{units}.h5")

In [ ]:
# Plot training loss for each configuration
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 4))
for units, losses in train_loss_per_unit:
    plt.plot(losses, label=f'LSTM Units {units}')
plt.title("Training Loss per Epoch for Different LSTM Sizes")
plt.xlabel("Epoch")
plt.ylabel("Loss (MSE)")
plt.legend()
plt.grid(True)
plt.savefig(f"{SAVE_DIR}/training_loss_per_epoch.png")
plt.show()


In [ ]:
# Evaluate over multiple windows and horizons
windows = [-90, -75, -60, -45, -30]  # Historical windows
horizons = [1, 5, 10, 15]            # Forecast lengths
window_size = 30

results = []

for win in windows:
    hist = df[:win]
    test_full = df[win:]

    # Prophet base model
    model = Prophet(
        growth='linear',
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=True,
        changepoint_prior_scale=0.05,
        seasonality_prior_scale=10.0,
        holidays_prior_scale=10.0,
        n_changepoints=25
    )
    model.add_seasonality(name='hourly', period=1, fourier_order=3)
    model.fit(hist)

    # Forecast with Prophet
    future = model.make_future_dataframe(periods=len(test_full), freq='min')
    forecast = model.predict(future)
    hist = hist.copy()
    hist['yhat'] = forecast.loc[:len(hist)-1, 'yhat'].values
    hist['residual'] = hist['y'] - hist['yhat']

    # Scale residuals
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    res_scaled = scaler.fit_transform(hist['residual'].values.reshape(-1, 1))

    # Create LSTM training data
    X, y_lstm = [], []
    for i in range(window_size, len(res_scaled) - max(horizons)):
        X.append(res_scaled[i - window_size:i])
        y_lstm.append(res_scaled[i:i + max(horizons)].flatten())
    X, y_lstm = np.array(X), np.array(y_lstm)

    # Tune LSTM model
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense
    from tensorflow.keras.callbacks import EarlyStopping

    best_model = None
    best_loss = float('inf')

    for units in [32, 64]:
        model_lstm = Sequential()
        model_lstm.add(LSTM(units, activation='tanh', input_shape=(X.shape[1], X.shape[2])))
        model_lstm.add(Dense(max(horizons)))
        model_lstm.compile(optimizer='adam', loss='mse')
        es = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
        history = model_lstm.fit(X, y_lstm, epochs=15, batch_size=32, verbose=0, callbacks=[es])
        if history.history['loss'][-1] < best_loss:
            best_loss = history.history['loss'][-1]
            best_model = model_lstm

    # Predict residuals from last window
    last_window = res_scaled[-window_size:].reshape(1, window_size, 1)
    pred_res_scaled = best_model.predict(last_window)
    pred_res = scaler.inverse_transform(pred_res_scaled.reshape(-1, 1)).flatten()

    # Prophet future forecast
    future = model.make_future_dataframe(periods=max(horizons), freq='min')
    forecast_future = model.predict(future).tail(max(horizons))
    yhat_prophet = forecast_future['yhat'].values

    # Actual values from full df
    y_true_all = df['y'].iloc[win:win + max(horizons)].values

    # Evaluate each horizon
    for h in horizons:
        y_pred_hybrid = yhat_prophet[:h] + pred_res[:h]
        y_true = y_true_all[:h]

        mae = mean_absolute_error(y_true, y_pred_hybrid)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred_hybrid))
        r2 = r2_score(y_true, y_pred_hybrid)
        mape = mean_absolute_percentage_error(y_true, y_pred_hybrid)
        accuracy = max(0, 100 * (1 - mape))

        results.append((abs(win), h, mae, rmse, r2, mape, accuracy))


In [ ]:
result_df = pd.DataFrame(results, columns=['Window (min ago)', 'Horizon (min)', 'MAE', 'RMSE', 'R2', 'MAPE', 'Accuracy (%)'])
result_df.to_csv(f"{SAVE_DIR}/hybrid_multiwindow_evaluation.csv", index=False)
result_df


In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(forecast_prophet['ds'], forecast_prophet['y'], label='Actual', marker='o')
plt.plot(forecast_prophet['ds'], forecast_prophet['yhat'], label='Prophet Only', marker='x')
plt.plot(forecast_prophet['ds'], forecast_prophet['yhat_lstm'], label='Hybrid (Prophet + LSTM)', marker='s')
plt.title('Forecast Comparison (Next 15 Minutes)')
plt.xlabel('Time')
plt.ylabel('Requests')
plt.legend()
plt.grid(True)
plt.savefig(f"{SAVE_DIR}/hybrid_forecast_plot.png")
plt.show()


In [ ]:
# 📊 Generate heatmap of Accuracy (%) by Window and Horizon
import seaborn as sns

heatmap_data = result_df.pivot(index="Window (min ago)", columns="Horizon (min)", values="Accuracy (%)")

plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_data, annot=True, fmt=".1f", cmap="YlGnBu", cbar_kws={'label': 'Accuracy (%)'})
plt.title("Hybrid Forecasting Accuracy (%) by Window and Horizon")
plt.ylabel("Window (Minutes Ago)")
plt.xlabel("Forecast Horizon (Minutes Ahead)")
plt.tight_layout()
plt.savefig(f"{SAVE_DIR}/hybrid_accuracy_heatmap.png")
plt.show()
